### Imports & Env Setup

In [2]:
%reload_ext autoreload
%autoreload 2
import sys
import os
from dotenv import load_dotenv
load_dotenv()
from datasets import load_dataset


import dspy
sys.path.append(os.path.abspath('../'))
from benchmarks import llama_mmlu_pro, leaderboard_mmlu_pro, llama_mmlu

### Configuration

In [3]:
NUM_THREADS = 48

FEW_SHOTS = 5

# See https://docs.litellm.ai/docs/providers/vllm for details
TASK_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1' , # or api_base ?
    api_key = "dummy",
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)
PROMPT_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1', # or api_base ?
    api_key = "dummy",

    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)

dspy.configure(lm=TASK_MODEL)

# replace this with llama_mmlu_pro or whatever
benchmark = llama_mmlu

# Without chain of thought:
# program = dspy.Predict(
#     benchmark.signature("")
# )

# With chain of thought:
program = dspy.ChainOfThought(
    benchmark.signature("You are a knowledgeable and insightful assistant designed to provide guidance on multiple-choice questions. Your role involves analyzing questions, assessing answer choices, and offering well-reasoned explanations to enhance understanding. By breaking down complex concepts, you help users develop critical thinking skills and improve their decision-making process. You strive to present information in a clear, structured manner while adapting to the user's level of expertise. Ultimately, your goal is to foster deeper comprehension and confidence in tackling multiple-choice assessments.") # put your initial system prompt here, or leave blank
)

evaluate = dspy.Evaluate(
    devset=[],
    max_errors = 500,
    metric=benchmark.metric,
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=True,
    return_all_scores=True,
    return_outputs=True,
    provide_traceback=True
)

### Load dataset

In [4]:
trainset, valset, testset = benchmark.datasets(
    train_size=0.1,
    validation_size=0.1,
)

len(trainset), len(valset), len(testset)

(1403, 1263, 11369)

### Baseline Benchmark

In [146]:
%%time
print("BASE PROMPT:\n", program.signature.instructions)

BASE PROMPT:
 
CPU times: user 348 μs, sys: 0 ns, total: 348 μs
Wall time: 283 μs


In [147]:
%%time
print("BEST EXAMPLES:\n", program.demos)

BEST EXAMPLES:
 []
CPU times: user 300 μs, sys: 16 μs, total: 316 μs
Wall time: 248 μs


In [148]:
%%time

print("Starting execution...")
# eval_subset_size = len(testset)
score, results, all_scores = evaluate(
    program,
    devset=testset[:40],
)

Starting execution...
  0%|                                                                                                                                                                                                                                            | 0/25 [11:06<?, ?it/s]

2025/01/30 00:13:12 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Find the value of the integral $\\int_S(\\nabla \\times \\mathbf{A}) \\cdot d \\mathbf{a}$ if the vector $\\mathbf{A}=y \\mathbf{i}+z \\mathbf{j}+x \\mathbf{k}$ and $S$ is the surface defined by the paraboloid $z=1-x^2-y^2$, where $z \\geq 0$.', 'options': {'A': '$2\\pi$', 'B': '$\\pi$', 'C': '$\\frac{\\pi}{2}$', 'D': '$4\\pi$', 'E': '$-\\frac{\\pi}{2}$', 'F': '$-2\\pi$', 'G': '$3\\pi$', 'H': '$0$', 'I': '$-\\pi$', 'J': '$-3\\pi$'}, 'answer': ''}) (input_keys={'options', 'question'}): 'list' object has no attribute 'items'
Stack trace:
Traceback (most recent call last):
  File "/home/justinai/.conda/envs/prompt-migration/lib/python3.10/site-packages/dspy/adapters/base.py", line 30, in __call__
    value = self.parse(signature, output, _parse_values=_parse_values)
  File "/home/justinai/.conda/envs/prompt-migration/lib/python3.10/site-packages/dspy/utils/callback.py", line 234, in wrapper
    


Average Metric: 23.00 / 39 (59.0%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 2965.01it/s]

2025/01/30 00:13:12 INFO dspy.evaluate.evaluate: Average Metric: 23.0 / 40 (57.5%)


,question,options,example_answer,reasoning,pred_answer,metric,answer
0,Two xylem plant cell types that provide support and conduct water ...,"{'A': 'parenchyma and companion cells', 'B': 'tracheids and vessel...",B,"To answer this question, we need to identify the two types of xyle...",B,✔️ [True],NaN
1,Calculate the drag force acting on a 0.75 ft × 7 ft smooth platewh...,"{'A': '6.9 lb_f', 'B': '14.2 lb_f', 'C': '11.3 lb_f', 'D': '12.4 l...",D,"To calculate the drag force acting on the plate, we can use the fo...",A,,NaN
2,For which of the following is it appropriate to use a census?,{'A': 'A two-tailed hypothesis test where the null hypothesis was ...,F,A census is appropriate when the goal is to collect data from ever...,J,,NaN
3,A housing corporation owned a tract of land and prepared a develop...,{'A': 'The community association has no authority to enforce the D...,E,The best argument for the landscaper is that the annual assessment...,F,,NaN
4,A population of students taking a basic genetics class had their b...,"{'A': 'p=0.415, q=0.2075, r=0.3133', 'B': 'p=0.2075, q=0.3133, r=0...",F,"To find the frequencies of the alleles I^A, I^B, and i, we can use...",D,,NaN
5,Hydrogen chloride (HCl) is used industrially in the manufacture of...,"{'A': '20.5 %', 'B': '19.3 %', 'C': '23.7 %', 'D': '16.8 %', 'E': ...",D,"To estimate the percent ionic character of the HCl bond, we can us...",B,,NaN
6,What is the smallest whole number that has a remainder of 1 when d...,"{'A': '72', 'B': '76', 'C': '67', 'D': '64', 'E': '57', 'F': '45',...",E,To find the smallest whole number that satisfies all the given con...,J,,NaN
7,We roll a fair 6-sided die 5 times. What is the probability that w...,"{'A': '\frac{25}{648}', 'B': '\frac{1000}{648}', 'C': '\frac{625}{...",,"To find the probability of getting a 6 in at most 2 of the rolls, ...",C,,NaN
8,A bird is lost in a 3 by 3 by 3 cubical maze. The bird flies from ...,"{'A': '1.95', 'B': '2.03', 'C': '1.85', 'D': '2.25', 'E': '2.10', ...",A,"To find the entropy rate of the random walk, we first need to unde...",A,✔️ [True],NaN
9,How do iconic concepts answer the question of how universals or ge...,"{'A': ""Iconic concepts represent 'universals' by dynamically creat...",F,"To answer this question, we need to consider how iconic concepts, ...",F,✔️ [True],NaN


CPU times: user 598 ms, sys: 8.98 ms, total: 607 ms
Wall time: 598 ms


In [12]:
import csv

with open("my_results.csv", mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # If `results` is a list of lists, just write rows:
    for row in results:
        writer.writerow(row)


In [13]:
import pandas as pd

df = pd.DataFrame([(i, str(pred), score) for i, (example, pred, score) in enumerate(results)],
                     columns=['Example_Index', 'Prediction', 'Score'])
print("\nResults DataFrame:")
print(df)


Results DataFrame:
    Example_Index  \
0               0   
1               1   
2               2   
3               3   
4               4   
5               5   
6               6   
7               7   
8               8   
9               9   
10             10   
11             11   
12             12   
13             13   
14             14   
15             15   
16             16   
17             17   
18             18   
19             19   
20             20   
21             21   
22             22   
23             23   
24             24   
25             25   
26             26   
27             27   
28             28   
29             29   
30             30   
31             31   
32             32   
33             33   
34             34   
35             35   
36             36   
37             37   
38             38   
39             39   
40             40   
41             41   
42             42   
43             43   
44             44   
45            

In [ ]:
comparison_df = pd.DataFrame([{
        'Question': example.inputs()['question'][:100] + '...' if len(example.inputs()['question']) > 100 else example.inputs()['question'],
        'Predicted Answer': prediction.answer if hasattr(prediction, 'answer') else str(prediction),
        'Correct Answer': example.answer,
        'Is Correct': '✔️' if score else '❌'
} for example, prediction, score in results])


csv_filename = 'prediction_results.csv'
comparison_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')  # utf-8-sig to handle special characters
print(f"\nResults saved to {csv_filename}")
    
pd.set_option('display.max_colwidth', None)
print("\nPredictions vs Actual Answers:")
print(comparison_df)

In [ ]:
%%time

print("Starting execution...")
evaluate(
    program,
    devset=testset,
)

### Optimize Subset + Evaluation

In [5]:
import logging
logging.getLogger('dspy').setLevel(logging.DEBUG)

In [7]:
%%time
subset_size = 200
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="light",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
    max_bootstrapped_demos=0,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/30 01:17:12 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 5
valset size: 100

2025/01/30 01:17:12 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/30 01:17:12 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2025/01/30 01:17:12 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


  0%|                                                                                                                                                                                                                                           | 0/200 [00:00<?, ?it/s]
2025/01/30 01:17:12 INFO dspy.teleprompt.mipro_optimizer_v2: Error generating few-shot examples: empty range for randrange() (1, 1, 0)
2025/01/30 01:17:12 INFO dspy.teleprompt.mipro_optimizer_v2: Running without few-shot examples.
2025/01/30 01:17:12 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/01/30 01:17:12 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.


Bootstrapped 0 full traces after 0 examples for up to 1 rounds, amounting to 0 attempts.
Bootstrapping set 4/5


2025/01/30 01:17:28 INFO dspy.teleprompt.mipro_optimizer_v2: 
Proposing instructions...

2025/01/30 01:17:46 INFO dspy.teleprompt.mipro_optimizer_v2: Proposed Instructions for Predictor 0:

2025/01/30 01:17:46 INFO dspy.teleprompt.mipro_optimizer_v2: 0: You are a knowledgeable and insightful assistant designed to provide guidance on multiple-choice questions. Your role involves analyzing questions, assessing answer choices, and offering well-reasoned explanations to enhance understanding. By breaking down complex concepts, you help users develop critical thinking skills and improve their decision-making process. You strive to present information in a clear, structured manner while adapting to the user's level of expertise. Ultimately, your goal is to foster deeper comprehension and confidence in tackling multiple-choice assessments.

2025/01/30 01:17:46 INFO dspy.teleprompt.mipro_optimizer_v2: 1: Analyze the given multiple-choice question and options, then provide a step-by-step reason

...


Average Metric: 89.00 / 99 (89.9%):  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 99/100 [00:21<00:01,  1.62s/it]

2025/01/30 01:21:11 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'A 30 mF capacitor has 6 millicoulombs of charge on each plate. The energy stored in the capacitor is most nearly', 'options': {'A': '5.4 × 10–10 J', 'B': '9.0 × 10–8 J', 'C': '0.6 J', 'D': '12.5 J'}, 'answer': 'C'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 89.00 / 99 (89.9%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:48<00:00,  2.05it/s]

2025/01/30 01:21:11 INFO dspy.evaluate.evaluate: Average Metric: 89.0 / 100 (89.0%)
2025/01/30 01:21:11 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 89.0
2025/01/30 01:21:11 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [88.0, 89.0]
2025/01/30 01:21:11 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 89.0
2025/01/30 01:21:11 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/30 01:21:11 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/30 01:21:11 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 89.0!



CPU times: user 4.02 s, sys: 932 ms, total: 4.95 s
Wall time: 3min 59s


In [8]:
%%time
print("BEST PROMPT:\n", optimized_program.signature.instructions)

BEST PROMPT:
 You are a knowledgeable and insightful assistant designed to provide guidance on multiple-choice questions. Your role involves analyzing questions, assessing answer choices, and offering well-reasoned explanations to enhance understanding. By breaking down complex concepts, you help users develop critical thinking skills and improve their decision-making process. You strive to present information in a clear, structured manner while adapting to the user's level of expertise. Ultimately, your goal is to foster deeper comprehension and confidence in tackling multiple-choice assessments.
CPU times: user 254 μs, sys: 22 μs, total: 276 μs
Wall time: 238 μs


In [ ]:
for module_name, module in optimized_program.__dict__.items():
    if hasattr(module, 'signature'):
        print(f"\nModule {module_name} instructions:")
        print(module.signature.instructions)

In [106]:
print("BEST EXAMPLES:\n", optimized_program.demos)

BEST EXAMPLES:
 []


In [51]:
import json

def example_to_dict(example):
    return {
        'question': example.question,
        'options': example.options,
        'answer': example.answer,
        'reasoning': example.reasoning,
        'input_keys': list(example.input_keys) if hasattr(example, 'input_keys') else None
    }

examples_json = [example_to_dict(example) for example in optimized_program.demos]
print("BEST EXAMPLES:\n", json.dumps(examples_json, indent=2))

BEST EXAMPLES:
 [
  {
    "question": "Dr. Ryan is a psychotherapist in a small town. She receives a phone call from a man she was sexually involved with several years ago. The relationship lasted only three months and ended amicably. The man is now married and is having problems unrelated to their past relationship. He has called to see if he can begin seeing Dr. Ryan in therapy. Dr. Ryan should:",
    "options": {
      "A": "see the man but disclose their past relationship to a supervisor or colleague for transparency.",
      "B": "deny the man's request without providing a reason.",
      "C": "see the man only if their relationship ended more than two years ago and she determines that it will not interfere with her objectivity.",
      "D": "see the man only if she feels she can maintain professional boundaries despite their past.",
      "E": "refer the man to a colleague.",
      "F": "continue therapy with the man but avoid discussing their past relationship.",
      "G": "see

In [94]:
%%time
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:40],
)

Average Metric: 37.00 / 40 (92.5%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 2826.92it/s]

2025/01/29 23:03:16 INFO dspy.evaluate.evaluate: Average Metric: 37 / 40 (92.5%)


,question,options,example_answer,example_reasoning,pred_reasoning,pred_answer,metric
0,For which of these two scenarios does the main character (who uses...,"{'A': 'Wrong, Wrong', 'B': 'Wrong, Not wrong', 'C': 'Not wrong, Wr...",A,"## Step 1: Evaluate the first scenario In the first scenario, the ...",To determine which scenario involves the main character doing some...,A,✔️ [True]
1,"Two lists, list1 and list2, contain the names of books found in tw...","{'A': 'newList ← Combine (listl, list2)\n newList ← Sort (newList)...",A,"To create newList, which contains the names of all books found in ...","To create newList, which contains the names of all books found in ...",A,✔️ [True]
2,How might the recent global economic crisis be viewed as a challen...,"{'A': 'Governmental cyberspace restrictions, in the form of censor...",C,## Step 1: Understanding the liberalist perspective The liberalist...,The liberalist perspective emphasizes the importance of free marke...,C,✔️ [True]
3,What term is used to represent unavoidable past costs that cannot ...,"{'A': 'Prime costs', 'B': 'Sunk costs', 'C': 'Opportunity costs', ...",B,## Step 1: Understanding the concept of sunk costs Sunk costs are ...,"To answer this question, we need to understand the concept of each...",B,✔️ [True]
4,Markson Co. traded a concrete-mixing truck with a book value of $1...,{'A': 'Does the book value of the asset given up exceed the fair v...,C,## Step 1: Understand the concept of commercial substance in asset...,To determine whether an exchange of assets has commercial substanc...,C,✔️ [True]
5,Which of the following substances is found in greater quantity in ...,"{'A': 'Carbon dioxide', 'B': 'Carbon monoxide', 'C': 'Nitrogen', '...",A,## Step 1: Understanding the context of the question The question ...,To determine which substance is found in greater quantity in exhal...,A,✔️ [True]
6,For which of these two scenarios does the main character (who uses...,"{'A': 'Wrong, Wrong', 'B': 'Wrong, Not wrong', 'C': 'Not wrong, Wr...",D,"## Step 1: Analyzing Scenario 1 In Scenario 1, the main character ...",To determine which scenario involves the main character doing some...,D,✔️ [True]
7,For which of these two scenarios does the main character (who uses...,"{'A': 'Wrong, Wrong', 'B': 'Wrong, Not wrong', 'C': 'Not wrong, Wr...",C,"## Step 1: Evaluate the first scenario In the first scenario, the ...",To determine which scenario involves the main character doing some...,C,✔️ [True]
8,"In the inherited disorder called Pompe disease, glycogen breakdown...","{'A': 'enzymes in the lysosomes', 'B': 'enzymes in the mitochondri...",A,## Step 1: Understanding Pompe Disease Pompe disease is an inherit...,Pompe disease is characterized by the accumulation of glycogen in ...,A,✔️ [True]
9,A state legislature has recently enacted an obscenity statute proh...,"{'A': 'The particular materials involved depicted normal, not devi...",B,To determine which of the given options would be most helpful to t...,To determine which of the given options would be most helpful to t...,B,✔️ [True]


CPU times: user 63.8 ms, sys: 8.1 ms, total: 71.9 ms
Wall time: 66.7 ms


## Medium Optimization

In [95]:
%%time
subset_size = 500
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="medium",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_bootstrapped_demos=0,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/29 23:05:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 25
valset size: 300

2025/01/29 23:05:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/29 23:05:39 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2025/01/29 23:05:39 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=25 sets of demonstrations...


Bootstrapping set 1/25
Bootstrapping set 2/25


  0%|▉                                                                                                                                                                                                                                                 | 2/500 [00:08<35:10,  4.24s/it]


Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 3/25


  1%|█▍                                                                                                                                                                                                                                              | 3/500 [00:00<00:00, 1029.78it/s]


...


Average Metric: 22.00 / 25 (88.0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:43<00:00,  1.74s/it]

2025/01/29 23:27:15 INFO dspy.evaluate.evaluate: Average Metric: 22 / 25 (88.0%)
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 88.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 7'].
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [92.0, 96.0, 100.0, 88.0, 88.0, 92.0, 84.0, 84.0, 92.0, 88.0, 96.0, 96.0, 84.0, 92.0, 88.0, 88.0, 84.0, 88.0, 80.0, 88.0, 84.0, 88.0, 92.0, 88.0, 88.0]
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [90.33, 88.0, 87.33]
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 90.33
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: =============================


2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Full Eval 3 =====
2025/01/29 23:27:15 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 92.0) from minibatch trials...



Average Metric: 267.00 / 300 (89.0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:30<00:00,  3.30it/s]

2025/01/29 23:28:46 INFO dspy.evaluate.evaluate: Average Metric: 267 / 300 (89.0%)
2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [90.33, 88.0, 87.33, 89.0]
2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 90.33
2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/29 23:28:46 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 90.33!



CPU times: user 18.3 s, sys: 3.02 s, total: 21.3 s
Wall time: 23min 6s


In [96]:
print("BEST PROMPT:\n", optimized_program.signature.instructions)

BEST PROMPT:
 You are a knowledgeable and insightful assistant designed to provide guidance on multiple-choice questions. Your role involves analyzing questions, assessing answer choices, and offering well-reasoned explanations to enhance understanding. By breaking down complex concepts, you help users develop critical thinking skills and improve their decision-making process. You strive to present information in a clear, structured manner while adapting to the user's level of expertise. Ultimately, your goal is to foster deeper comprehension and confidence in tackling multiple-choice assessments.


In [98]:
print("BEST EXAMPLES:\n", optimized_program.demos)

BEST EXAMPLES:
 []


In [48]:
examples_json = [example.to_dict() for example in optimized_program.predict.demos]
print("BEST EXAMPLES:\n", json.dumps(examples_json, indent=2))

AttributeError: 'ChainOfThought' object has no attribute 'predict'

In [99]:
%%time
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:500],
)

Average Metric: 453.00 / 500 (90.6%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [01:42<00:00,  4.90it/s]

2025/01/29 23:32:56 INFO dspy.evaluate.evaluate: Average Metric: 453 / 500 (90.6%)


,question,options,example_answer,example_reasoning,pred_reasoning,pred_answer,metric
0,For which of these two scenarios does the main character (who uses...,"{'A': 'Wrong, Wrong', 'B': 'Wrong, Not wrong', 'C': 'Not wrong, Wr...",A,"## Step 1: Evaluate the first scenario In the first scenario, the ...",To determine which scenario involves the main character doing some...,A,✔️ [True]
1,"Two lists, list1 and list2, contain the names of books found in tw...","{'A': 'newList ← Combine (listl, list2)\n newList ← Sort (newList)...",A,"To create newList, which contains the names of all books found in ...","To create newList, which contains the names of all books found in ...",A,✔️ [True]
2,How might the recent global economic crisis be viewed as a challen...,"{'A': 'Governmental cyberspace restrictions, in the form of censor...",C,## Step 1: Understanding the liberalist perspective The liberalist...,The liberalist perspective emphasizes the importance of free marke...,C,✔️ [True]
3,What term is used to represent unavoidable past costs that cannot ...,"{'A': 'Prime costs', 'B': 'Sunk costs', 'C': 'Opportunity costs', ...",B,## Step 1: Understanding the concept of sunk costs Sunk costs are ...,"To answer this question, we need to understand the concept of each...",B,✔️ [True]
4,Markson Co. traded a concrete-mixing truck with a book value of $1...,{'A': 'Does the book value of the asset given up exceed the fair v...,C,## Step 1: Understand the concept of commercial substance in asset...,To determine whether an exchange of assets has commercial substanc...,C,✔️ [True]
...,...,...,...,...,...,...,...
495,"A man is charged with murder. During the trial, defense counsel of...","{'A': 'not hearsay.', 'B': 'hearsay, but admissible as an admissio...",D,"To answer this question, let's break it down step by step: ## Step...",The testimony in question involves a statement made by a man on de...,D,✔️ [True]
496,"Two men held-up a liquor store in a city. During the robbery, one ...","{'A': 'granted, because the prosecutor is constitutionally require...",B,"To answer this question, we need to consider the legal implication...",The defendant's motion to dismiss the indictment due to the delay ...,C,
497,Which vitamins are important in lowering circulating homocysteine ...,"{'A': 'Vitamin D', 'B': 'Vitamin C', 'C': 'Vitamin A', 'D': 'Folat...",D,## Step 1: Understanding the role of vitamins in homocysteine leve...,"To answer this question, we need to consider the role of vitamins ...",D,✔️ [True]
498,"This question refers to the following information. ""The greatest c...",{'A': 'African nations will not achieve independence without unity...,D,## Step 1: Understand the context of Nkrumah's statement Nkrumah e...,"To answer this question, we need to understand the context and the...",D,✔️ [True]


CPU times: user 4.91 s, sys: 587 ms, total: 5.49 s
Wall time: 1min 42s


## Heavy Optimization

In [ ]:
%%time
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="heavy",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset,
    valset=valset,
)

In [ ]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

In [ ]:
print("BEST EXAMPLES:\n", optimized_program.predict.demos)

In [ ]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset,
    display_table=False,
)